In [2]:
import requests
import pandas as pd
from config import api_key

In [153]:
pages = list(range(1,500))

movie_df = pd.DataFrame()



for year in range(1945, 2000):
    x = 0

    for page in pages:
        movie_df_length = len(movie_df)
        movie_data = pd.DataFrame(requests.get(f'https://api.themoviedb.org/3/discover/movie?api_key={api_key}&primary_release_date.gte={year}-01-01&primary_release_date.lte={year}-12-31&vote_count.gte=0&vote_average.gte=3.9&sort_by=release_date.asc&page={page}').json()['results'])
        movie_df = movie_df.append(movie_data)

        x += 1
        
        if len(movie_df) == movie_df_length:
            break

    print(year,x)

movie_df.reset_index(drop = True, inplace=True)
movie_df.drop(columns=['adult','backdrop_path','original_language','original_title','poster_path','video'], inplace=True)


movie_df.head()

1945 32
1946 34
1947 35
1948 39
1949 41
1950 44
1951 44
1952 42
1953 46
1954 45
1955 51
1956 51
1957 55
1958 56
1959 55
1960 54
1961 57
1962 59
1963 57
1964 60
1965 63
1966 66
1967 71
1968 73
1969 77
1970 77
1971 77
1972 82
1973 80
1974 78
1975 70
1976 74
1977 74
1978 77
1979 82
1980 79
1981 81
1982 80
1983 82
1984 84
1985 87
1986 89
1987 97
1988 101
1989 99
1990 100
1991 101
1992 95
1993 99
1994 100
1995 107
1996 107
1997 111
1998 118
1999 129


,genre_ids,id,overview,popularity,release_date,title,vote_average,vote_count
0,"[12, 35, 10749, 53, 10752]",22584,A Martinique charter boat skipper gets mixed u...,7.049,1945-01-20,To Have and Have Not,7.5,345
1,"[18, 10751]",17641,Mi Taylor is a young wanderer and opportunist ...,5.736,1945-01-26,National Velvet,7.4,77
2,"[18, 36]",9797,"Sergei Eisenstein's final film, this is the fi...",7.732,1945-01-20,"Ivan the Terrible, Part I",7.4,118
3,"[10402, 16]",48583,An animated safety film adapts the nursery rhy...,0.600,1945-01-01,The Three Blind Mice,5.0,5
4,"[14, 878, 10752]",126077,An American businessman returns from a hunting...,0.600,1945-01-01,Strange Holiday,6.7,3


In [154]:
len(movie_df)

78922

In [156]:
movie_df_copy = movie_df.loc[movie_df.astype(str).drop_duplicates(subset='title').index]

In [157]:
movie_df_copy = pd.DataFrame(movie_df_copy)
movie_df_copy.count()

genre_ids       68845
id              68845
overview        68845
popularity      68845
release_date    68845
title           68845
vote_average    68845
vote_count      68845
dtype: int64

In [158]:
movie_df_copy.loc[movie_df_copy['title']=='The Jazz Singer']

,genre_ids,id,overview,popularity,release_date,title,vote_average,vote_count
5988,"[10402, 18]",216925,As Jerry Golding scales the heights of show bu...,1.059,1953-02-14,The Jazz Singer,5.8,5


In [159]:
keyword_df = pd.read_csv('resources/BC_keywords.csv')
antikeyword_df = pd.read_csv('resources/BC_antikeywords.csv')

#Change Column name
keyword_df.rename({'Topic/Keyword':'keyword'}, axis=1, inplace=True)

key_list = keyword_df.keyword.to_list()
antikey_list = antikeyword_df.BC_anti_keywords.to_list()
key_list

['Neanderthals',
 'Prehistoric',
 'Mamoth',
 'pre-historic',
 'Jomon',
 'Naqada',
 'Minoan',
 'Hor-Aha',
 'Harappa',
 'Egypt',
 'Menes',
 'Pharoah',
 'Narmer',
 'Scorpion King',
 'Troy ',
 'Jerusalem',
 'Djoser',
 'Pyramids',
 'Khufu',
 'Giza',
 'Stonehenge',
 'Xia Dynasty',
 'Yu the Great',
 'Babylon',
 'Hammurabi',
 'Tigris',
 'Shang Dynasty',
 'Assyria',
 'Nefertiti',
 'Tut',
 'Achilles',
 'Trojan war',
 'Moses',
 'Ramses',
 'Dorians',
 'Hittite',
 'Petra',
 'Samson',
 'Zhou Dynasty',
 'Sushruta',
 'Sparta',
 'Dionysus',
 'Bathsheba',
 'Solomon',
 'The Temple Mount',
 'Carthage',
 'Macedonia',
 'Olympia',
 'Rome',
 'Romulus',
 'Anaximander',
 'Pythagoras',
 'Xenophanes',
 'Battle of Hyrba',
 'Battle of the Persian Border',
 'Confucious',
 'Nicolaus of Damascus',
 'Cyrus the Great',
 'Cyrus',
 'Temple of Artemis',
 'Persia',
 'Persian',
 'Xerxes',
 'Leonidas',
 'Hippias',
 'Persian Wars',
 'Battle of Thermopylae',
 'Thermopylae',
 'Battle of Salamis',
 'Salamis',
 'Archidamus',
 'Soc

In [161]:
column_list = movie_df.columns.to_list()

revised_movie_df = pd.DataFrame(columns=column_list)

keyword_column = []
dates_column = []
i = 0

for movie in movie_df_copy.overview:
    key_holder = []
    date_holder = []
    j = 0
    k = 0

    for keyword in key_list:
        if keyword in movie:
            date_holder.append(keyword_df['Start Date'][j])
            key_holder.append(keyword)
            if len(key_holder) == 1 and k == 0:
                try:
                    revised_movie_df = revised_movie_df.append(movie_df_copy.iloc[[i]])
                    k += 1
                except KeyError:
                    print('KeyError found')
        j += 1
    if len(key_holder) != 0:
        keyword_column.append(key_holder)
        dates_column.append(date_holder)
    i += 1
revised_movie_df['keyword'] = keyword_column
revised_movie_df['est_date'] = dates_column

revised_movie_df.reset_index(drop = True, inplace=True)

i_2 = 0
drop_list = []

for movie in revised_movie_df.overview:
    for antikey in antikey_list:   
        if antikey in movie:
            drop_list.append(i_2) 
            break      
    i_2 += 1
revised_movie_df = revised_movie_df.drop(drop_list,axis=0)

revised_movie_df.reset_index(drop = True, inplace=True)

revised_movie_df

,genre_ids,id,overview,popularity,release_date,title,vote_average,vote_count,keyword,est_date
0,"[36, 10749]",364684,A Romeo & Juliet-esque tale set in Uzbekistan.,0.621,1945-10-21,Takhir and Zukhra,8.0,1,[Rome],[-753]
1,"[18, 36, 35]",31561,The aging Caesar finds himself intrigued by th...,6.982,1945-12-11,Caesar and Cleopatra,6.3,36,[Egypt],[-3150]
2,[18],43469,"At a track near Rome, shoeshine boys are watch...",5.014,1946-04-27,Shoeshine,7.8,167,[Rome],[-753]
3,[12],43464,Daredevil sailor Sinbad embarks on a voyage ac...,3.564,1947-01-13,Sinbad the Sailor,6.2,28,"[Persia, Alexander the Great]","[-550, -356]"
4,[35],116259,"Set in ancient Egypt, the stooges run a used c...",0.931,1948-11-04,Mummy's Dummies,7.3,3,[Egypt],[-3150]
...,...,...,...,...,...,...,...,...,...,...
583,"[35, 10749, 18]",62958,Naomi Shimizu is a young career woman working ...,1.241,1999-08-21,Messengers,5.6,10,[Rome],[-753]
584,"[35, 18]",22908,"Two twelve-year-old boys, Romeo and Gavin, und...",2.370,1999-08-23,A Room for Romeo Brass,6.9,40,[Rome],[-753]
585,"[10752, 28, 10770]",116333,When an elite eight-man British SAS team is dr...,2.536,1999-10-07,Bravo Two Zero,5.6,25,"[Persia, Persian]","[-550, -550]"
586,[10751],299856,"Besides the stray dog Mingus, 11 year old Soph...",0.662,1999-10-14,The Ball,4.0,1,[Rome],[-753]


In [162]:
revised_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genre_ids     588 non-null    object 
 1   id            588 non-null    object 
 2   overview      588 non-null    object 
 3   popularity    588 non-null    float64
 4   release_date  588 non-null    object 
 5   title         588 non-null    object 
 6   vote_average  588 non-null    float64
 7   vote_count    588 non-null    object 
 8   keyword       588 non-null    object 
 9   est_date      588 non-null    object 
dtypes: float64(2), object(8)
memory usage: 46.1+ KB


In [140]:
desired_movies = pd.read_csv('resources/desired_movies.csv')

id_list = desired_movies['id'].tolist()

movie_id = []
title_name = []
vote_count = []
vote_average = []

for movie in id_list:
    id = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['id']
    title = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['title']
    count = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['vote_count']
    average = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['vote_average']
    
    movie_id.append(id)
    title_name.append(title)
    vote_count.append(count)
    vote_average.append(average)

desired_movie_df = pd.DataFrame(
    {'movie_id':movie_id,
    'title_name': title_name,
    'vote-count':vote_count,
    'vote_average':vote_average
    })

desired_movie_df.reset_index(drop = True, inplace=True)

In [14]:
genres = requests.get(f'https://api.themoviedb.org/3/genre/movie/list?api_key={api_key}&language=en-US').json()
genre_df = pd.DataFrame(genres)
genre_df

,genres
0,"{'id': 28, 'name': 'Action'}"
1,"{'id': 12, 'name': 'Adventure'}"
2,"{'id': 16, 'name': 'Animation'}"
3,"{'id': 35, 'name': 'Comedy'}"
4,"{'id': 80, 'name': 'Crime'}"
5,"{'id': 99, 'name': 'Documentary'}"
6,"{'id': 18, 'name': 'Drama'}"
7,"{'id': 10751, 'name': 'Family'}"
8,"{'id': 14, 'name': 'Fantasy'}"
9,"{'id': 36, 'name': 'History'}"
